In [ ]:
#infinite_stairs를 정리해서 제대로 만듬
#게임은 좌측화면 맨위 640 * 320 인가로 띄웠음.

In [114]:
from PIL import ImageGrab
import numpy as np
import cv2
from time import sleep
import win32com.client
import pyautogui
from time import time
from keras.layers import Dense, Conv2D , Activation , Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from collections import deque
import random
from keras import backend as K

In [115]:
class GameState:
    def __init__(self):
        pass
    
    def grab_screen(self, _driver = None): #전체화면 받아옴
        screen =  np.array((ImageGrab.grab(bbox=(10,30,350,650))) )
        image = self.process_img(screen)
        return image

    def process_img(self,image): #resize를 절반으로해줌
        image = cv2.resize(image, (0,0), fx = 0.11, fy = 0.2) 
        image = cv2.Canny(image, threshold1 = 120, threshold2 = 250)
        #print(image.shape)
        return  image 

    def grab_train(self): #학습할곳만 받아옴 (40x 32로함)
        screen =  np.array((ImageGrab.grab(bbox=(10,300,300,500)))) 
        image = self.process_img(screen)
        return image

    def real_grab_screen(self,_driver = None):

        screen =  np.array((ImageGrab.grab(bbox=(10,30,350,650))) )
        image = self.real_process_img(screen)#processing image as required
        return image

    def real_process_img(self,image):
        image = cv2.Canny(image, threshold1 = 100, threshold2 = 200)
        return  image 
    
    def start_button_click(self): #start_button이 있다면 클릭해줌
        large_image = self.real_grab_screen()
        small_image = cv2.imread("./image_video/start_button.PNG",0)
        small_image = self.real_process_img(small_image)
        if self.exists(large_image, small_image,0.7):
            w,h = small_image.shape[::-1]
            result = cv2.matchTemplate(large_image, small_image, cv2.TM_CCOEFF_NORMED)
            min_val,max_val,min_loc,max_loc = cv2.minMaxLoc(result)
            top_left = max_loc
            pyautogui.click(top_left[0]+w/2,top_left[1]+h/2)
            pyautogui.moveTo(100,100)
    def exists(self, image, template, thresh):
        digit_res = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
        loc = np.where(digit_res >= thresh)

        if len(loc[-1]) == 0:
            return False

        for pt in zip(*loc[::-1]):
            if digit_res[pt[1]][pt[0]] == 1:
                return False
        return True

    def up_button_exists(self): #up_button 이 있다면 게임이 안끝난것.
        large_image = self.real_grab_screen()
        small_image = cv2.imread("./image_video/up_button.PNG",0)
        small_image = self.real_process_img(small_image)

        return not (self.exists(large_image,small_image,0.7))
    
    def OKButton_exists(self):
        large_image = self.real_grab_screen()
        small_image = cv2.imread("./image_video/OK.PNG",0)
        small_image = self.real_process_img(small_image)
        
        return self.exists(large_image,small_image,0.7)
    
    def OK_clear(self):
        if self.OKButton_exists():
            large_image = self.real_grab_screen()
            small_image = cv2.imread("./image_video/OK.PNG",0)
            small_image = self.real_process_img(small_image)
            w,h = small_image.shape[::-1]
            result = cv2.matchTemplate(large_image, small_image, cv2.TM_CCOEFF_NORMED)
            min_val,max_val,min_loc,max_loc = cv2.minMaxLoc(result)
            top_left = max_loc
            pyautogui.click(top_left[0]+w/2,top_left[1]+h)
            sleep(1)
            pyautogui.click(370,570)
            sleep(1)
            pyautogui.moveTo(800,800)
        else :
            pass
    
    def bonus_clear(self):
        if self.bonus_exists():
            pyautogui.click(300,330)
            sleep(1)
            pyautogui.click(170,480)
            sleep(1)
            pyautogui.click(370,570)
            sleep(1)
            pyautogui.moveTo(800,800)
        
    def bonus_exists(self):
        large_image = self.real_grab_screen()
        small_image = cv2.imread("./image_video/bonus.PNG",0)
        small_image = self.real_process_img(small_image)
        
        return self.exists(large_image,small_image,0.7)

In [130]:
pyautogui.moveTo(170,550)
#sleep(5)
#pyautogui.moveTo(370,570)

In [116]:
class Agent:
    def __init__(self):
        #self.run()
        self.shell = win32com.client.Dispatch("WScript.Shell")
        self.epsilon = 1. #e
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.action_size = 2
        self.learning_rate = 0.0001
        self.gamma = 0.95
        self.target_model = self.build_model()
        self.main_model = self.build_model()
        self.memory = deque(maxlen =2000)
        
        self.image_channels = 4 #4개의 채널
        self.img_rows = 0 #이 세개 왜 build_model에서 찾지못함?
        self.img_cols = 0
        self.set_frame()
        
    def set_frame(self):
        self.img_rows = 40
        self.img_cols = 32
        
    def run(self):
        self.shell.SendKeys('l')
        sleep(0.005)
        return 0
    
    def turn(self):
        self.shell.SendKeys('a')
        sleep(0.005)
        return 1
    
    def act_thinking(self,state):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay  
            
        if np.random.rand() <= self.epsilon:     
            return random.randrange(self.action_size)
        
        act_values = self.main_model.predict(state)
        return np.argmax(act_values[0])
    
    def act(self,state):
        res = self.act_thinking(state)
        if res == 1:
            self.turn()
            return 1
        else :
            self.run()
            return 0
    def _huber_loss(self, target, prediction):
        # sqrt(1+error^2)-1
        error = prediction - target
        return K.mean(K.sqrt(1+K.square(error))-1, axis=-1)
    
    def build_model(self):
        self.img_rows = 40
        self.img_cols = 32
        self.img_channels = 4
        model = Sequential()
        model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='same',input_shape=\
                         (self.img_cols, self.img_rows, self.img_channels)))  #32*40*4
        model.add(Activation('relu'))
        model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same'))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dense(self.action_size))
        adam = Adam(lr=self.learning_rate)
        model.compile(loss=self._huber_loss,optimizer=adam)
        
        return model
    
    def remember(self, state, action, rewar, next_state, done):
        self.memory.append((state,action,reward,next_state,done))
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)

        #target_model
        #main_model
        for state, action, reward, next_state, done in minibatch:
            Q = self.main_model.predict(state)
            #print(Q,"Q")
            #print(Q[0][action],'Q[0][action]')
            
            if done:
                Q[0][action] = reward
                
            else :
                t = self.target_model.predict(next_state)[0]
                #print(t,"t")
                #print(reward,'reward')
                Q[0][action] = reward + self.gamma * np.amax(t)
            
            self.main_model.fit(state,Q,epochs=1,verbose=0)
            
    def save_model(self,name):
        agent.main_model.save_weights(name)
    
    def load_model(self,name):
        agent.main_model.load_weights(name)
        agent.target_model.load_weights(name)

In [136]:
agent.save_model("infinite_stairs_ver_02_epochs_500.h5")

In [117]:
def get_score(start_time):
    score = time()-start_time
    return score

In [122]:
score_board = []

In [121]:
agent.load_model('infinite_stairs_ver_01_epochs_300.h5')

In [120]:
episodes = 100
batch_size= 32


game = GameState()
agent = Agent()


In [107]:
yes = time()

In [123]:
episodes= 300

In [135]:
def start():
    for e in range(episodes):

        game.start_button_click()
        agent.run()
        start_time = time()
        state = game.grab_train()
        state = np.stack((state,state,state,state),\
                         axis=2).reshape(1, agent.img_cols, agent.img_rows,agent.img_channels)

        done=False

        for time_t in range(1000):

            action = agent.act(state)
            done = game.up_button_exists()

            reward = get_score(start_time)
            if done:
                reward = -10
            if (time_t >= 50):
                reward = 100
            #print(reward)
            next_state = game.grab_train()
            next_state = np.stack((next_state,next_state,next_state,next_state),\
                         axis=2).reshape(1, agent.img_cols, agent.img_rows,agent.img_channels)
            agent.remember(state, action, reward, next_state,done)
            state = next_state
            if done:
                sleep(2)
                game.OK_clear()
                game.bonus_clear()
                agent.target_model.set_weights(agent.main_model.get_weights())
                score_board.append(time_t)    
                print("episode: {}/{}, score: {}, e: {:.4}"
                        .format(e, episodes, time_t, agent.epsilon))
                if len(score_board) > 3 :
                    if score_board[-1] == 0 and score_board[-2] == 0 and score_board[-3] == 0 and score_board[-4] == 0:
                        pyautogui.click(170,550)
                        sleep(5)
                        pyautogui.click(370,570)
                break

            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
start()

episode: 0/300, score: 19, e: 0.009921
episode: 1/300, score: 20, e: 0.009921
episode: 2/300, score: 4, e: 0.009921
episode: 3/300, score: 28, e: 0.009921
episode: 4/300, score: 12, e: 0.009921
episode: 5/300, score: 44, e: 0.009921
episode: 6/300, score: 32, e: 0.009921
episode: 7/300, score: 26, e: 0.009921
episode: 8/300, score: 26, e: 0.009921
episode: 9/300, score: 42, e: 0.009921
episode: 10/300, score: 25, e: 0.009921
episode: 11/300, score: 45, e: 0.009921
episode: 12/300, score: 32, e: 0.009921
episode: 13/300, score: 53, e: 0.009921
episode: 14/300, score: 44, e: 0.009921
episode: 15/300, score: 50, e: 0.009921
episode: 16/300, score: 26, e: 0.009921
episode: 17/300, score: 25, e: 0.009921
episode: 18/300, score: 33, e: 0.009921
episode: 19/300, score: 48, e: 0.009921
episode: 20/300, score: 34, e: 0.009921
episode: 21/300, score: 35, e: 0.009921
episode: 22/300, score: 13, e: 0.009921
episode: 23/300, score: 13, e: 0.009921
episode: 24/300, score: 52, e: 0.009921
episode: 25

KeyboardInterrupt: 

In [ ]:
'''
class MC:
    def __init__(self):
        self.game = GameState()
        self.agent = Agent()
    def start(self):
        self.game.start_button_click()
        self.agent.run()
        start_time = time()
  
'''

In [ ]:
episodes = 1000
batch_size= 32
done=False

if __name__ == '__main__':
    game = GameState()
    agent = Agent()
    game.start_button_click()
    agent.run()
    start_time = time()
    '''
        action = predict하고
        
        end_time = time() 구함
        if done : 
    '''

    

In [112]:
agent.main_model.save_model

In [113]:
agent.main_model.save_weights("infinite_stairs_ver_01_epochs_300.h5")